![](img/A_slicing_structure_and_its_representations.png)
 > [A genetic algorithm approach to an integrated problem of shelf space design and item allocation](https://linkinghub.elsevier.com/retrieve/pii/S0360835208002209)

![A genetic algorithm for the two-dimensional knapsack problem with rectangular pieces](img/Layer_structure_of_a_packing_plan.png)
> [A genetic algorithm for the two-dimensional knapsack problem with rectangular pieces](https://onlinelibrary.wiley.com/doi/10.1111/j.1475-3995.2009.00701.x)

In [1]:
import random
import numpy as np
import matplotlib

import matplotlib.pyplot as plt

from deap import base, creator, tools, algorithms
from typing import List

plt.rcParams['figure.dpi'] = 150

class Operator:
    HORIZONTAL_CUT = "+" 
    VERTICAL_CUT   = "*"
    
    OPERATOR_LST = [HORIZONTAL_CUT, VERTICAL_CUT]
    
    @staticmethod
    def getRandom():
        return random.choice(OPERATOR_LST)
    
class Node:
    def __init__self(self, value):
        this.value = value
        
    def isLeaft(self):
        return False
    

class InternalNode(Node):
    def __init__(self, value, left, right):
        super.__init__(value)
        self.left = left
        self.right = right
        
class Leaf(Node):
    def __init__(self, value):
        super.__init__(value)
    
    def isLeaf(self):
        return True

        
class Rectangle:
    def __init__(self, width=0, height=0, x=0, y=0, ident=1):
        self.width = width
        self.height = height
        self.x = x
        self.y = y
        self.ident = ident
    
    def collides(self, oth):
        return  not(       (oth.y >= self.y+self.height)\
                        or (oth.x >= self.x+self.width)\
                        or (oth.y+oth.height <= self.y)\
                        or (oth.x+oth.width <= self.x))
    def __str__(self):
        return f'{self.ident}: ({self.width},{self.height},{self.x},{self.y}) whxy'
    
class Individual:
    def __init__(self, tree: Node):
        self.tree = tree


###########################################################################################

def generate_random_individual(w: int, h: int, recs: List[Rectangle], max_depth=6) -> Individual:
    root = Node(Operator.getRandom())
    root.left  = _generate_random_tree(curr_depth=1, max_depth=max_depth, parent=root)
    root.right = _generate_random_tree(curr_depth=1, max_depth=max_depth, parent=root)

    
def _generate_random_tree(w: int, h: int, curr_depth: int, max_depth: int, parent: Node) -> Node:
    if curr_depth >= max_depth\
        or random.uniform(0, 1) > (curr_depth/(max_depth*1.2))\
        or w <= 1 or h <=1:
            return _generate_terminal_tree(w,h)
    if random.uniform(0,1) > 0.5:
        return _generate_non_terminal_tree(w, h, curr_depth, max_depth)

def _generate_non_terminal_tree(w: int, h: int, curr_depth: int, max_depth: int):
    op = Operator.getRandom()
    tree = generate_operand(w, h, )
    x.left  = generate_operand(w, h, curr_depth+1, max_depth, x)
    x.right = generate_random_tree(w, h, curr_depth+1, max_depth, x)
    
def _generate_terminal_tree(w: int, h: int) -> Node:
    op = Operator.getRandom()
    return InternalNode(value=op,
                left =_generate_operand(w, h, op),
                right=_generate_operand(w, h, op))
    

def _generate_operand(w: int, h: int, op: Operator) -> Node:
        if op == Operator.HORIZONTAL_CUT:
            return Leaf(random.randint(1, h-1))
        else:
            return Leaf(random.randint(1, w-1))
        

###########################################################################################

def draw_rectangles(recs: List[Rectangle], strip_width: int, show_ident=True) -> None:
    def to_matplotlib_rec(rec):
        return matplotlib.patches.Rectangle(
            (rec.x, rec.y),
            width=rec.width,height=rec.height,
            edgecolor='blue',
            linewidth=1.3,
            fill=False,
        )
    fig, ax = plt.subplots()

    for rec in recs:
        ax.add_patch(to_matplotlib_rec(rec))
        if show_ident:
            ax.text(rec.x+rec.width/2, rec.y+rec.height/2, rec.ident, fontsize=8)


    mult = 1.2
    max_rectangle_height = calc_max_rectangle_height(recs)
    ax.set_xlim(left=-1, right=mult*strip_width)
    ax.set_ylim(bottom=0, top=mult*max_rectangle_height)
    ax.axvline(x=strip_width, color='red', linestyle='--')
    ax.axvline(x=0, color='red', linestyle='--')
    ax.set_title(f'maximum height={max_rectangle_height}', fontsize=8)
    
    plt.show()